Antes de inciar a regressão linear e polinomial, eu primeiro fiz a reconstrução.  

Primeiro selecionei as colunas em string e realizei a codificação de dummie, filtrei as colunas de meu interesse que possuiam valores nulos, e por fim transformei os valores float em int.

Retirei o R2 de cada coluna de meu interesse pra dar inicío aos meus testes e treinos.


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats

df = pd.read_csv('/home/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [55]:
df1 = df.copy()

In [56]:
df2 = df.copy()

In [57]:
df2['dummy_gender'] = df2.apply( lambda row: 1 if row['sex']=='male' else 0 , axis=1)
df2['dummy_smoker'] = df2.apply( lambda row: 1 if row['smoker']=='yes' else 0 , axis=1)
df2['dummy_region'] = df2.apply( lambda row: 1 if row['region']=='southeast' else 0 , axis=1)
df2['dummy_region'] = df2.apply( lambda row: 2 if row['region']=='southwest' else 0 , axis=1)
df2['dummy_region'] = df2.apply( lambda row: 3 if row['region']=='northeast' else 0 , axis=1)
df2['dummy_region'] = df2.apply( lambda row: 4 if row['region']=='northwest' else 0 , axis=1)
df2.head()
df2

,age,sex,bmi,children,smoker,region,charges,dummy_gender,dummy_smoker,dummy_region
0,19,female,27.900,0,yes,southwest,16884.92400,0,1,0
1,18,male,33.770,1,no,southeast,1725.55230,1,0,0
2,28,male,33.000,3,no,southeast,4449.46200,1,0,0
3,33,male,22.705,0,no,northwest,21984.47061,1,0,4
4,32,male,28.880,0,no,northwest,3866.85520,1,0,4
...,...,...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,1,0,4
1334,18,female,31.920,0,no,northeast,2205.98080,0,0,0
1335,18,female,36.850,0,no,southeast,1629.83350,0,0,0
1336,21,female,25.800,0,no,southwest,2007.94500,0,0,0


In [58]:
filtered_df = df2[df2['age'].notnull() & df2['bmi'].notnull() & df2['charges'].notnull()]

In [59]:
df2 = df2.apply(pd.to_numeric, errors='coerce')

In [60]:
x = filtered_df[['dummy_gender', 'dummy_smoker', 'dummy_region', 'children', 'charges', 'bmi']].to_numpy()
y = filtered_df['age'].to_numpy()

model_age = LinearRegression()
model_age.fit(x, y)

r2_score = model_age.score(x, y)
r2_score_arredondado = round(r2_score, 2)

print(f"R² (coeficiente de determinação) do modelo: {r2_score_arredondado}")

R² (coeficiente de determinação) do modelo: 0.27


In [61]:
x = filtered_df[['dummy_gender', 'dummy_smoker', 'dummy_region', 'children', 'charges', 'age']].to_numpy()
y = filtered_df['bmi'].to_numpy()

model_age = LinearRegression()
model_age.fit(x, y)

r2_score = model_age.score(x, y)
r2_score_arredondado = round(r2_score, 2)

print(f"R² (coeficiente de determinação) do modelo: {r2_score_arredondado}")

R² (coeficiente de determinação) do modelo: 0.12


In [62]:
x = filtered_df[['dummy_gender', 'dummy_smoker', 'dummy_region', 'children', 'bmi', 'age']].to_numpy()
y = filtered_df['charges'].to_numpy()

model_age = LinearRegression()
model_age.fit(x, y)

r2_score = model_age.score(x, y)
r2_score_arredondado = round(r2_score, 2)

print(f"R² (coeficiente de determinação) do modelo: {r2_score_arredondado}")

R² (coeficiente de determinação) do modelo: 0.75


In [63]:
# df.head()
print(df.shape)
# Dividir minha base total em teste e treino
df2_train = df.sample(200)

# retiro da base principal as observações que utilizo na base de treino
df2 = df2.drop(df2_train.index)

# Crio uma lista com 10 amostras de 50 observações cada a partir
# das 200 observações que sobraram após a retirada das observações
# de treino. HAVERÁ REPETIÇÃO
samples = []
for i in range(10):
  samples.append(df.sample(50))

(1338, 7)


In [64]:
from sklearn.model_selection import train_test_split

# Supondo que `df2` seja o seu DataFrame principal
df2_train, df2_test = train_test_split(df2, test_size=0.2, random_state=42)


In [65]:
# Criei minhas variáveis de treino
X_train = df2_train[['dummy_smoker']].to_numpy()
Y_train = df2_train['charges'].to_numpy()

# Criei meu modelo de Regressão Linear
model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
score_linear = model_linear.score(X_train, Y_train)
score_linear

0.6250613522379534

In [66]:
# Criei meu modelo de Regressão Polinomial
poly_features = PolynomialFeatures(degree=3)
X_poly_train = poly_features.fit_transform(X_train)
model_poly = LinearRegression()
model_poly.fit(X_poly_train, Y_train)
score_polynomial = model_poly.score(X_poly_train, Y_train)
score_polynomial

0.6235990126892539

In [67]:
# Atualizar samples com a nova coluna
samples = [df2.sample(50) for _ in range(10)]


In [68]:
# Começar a testar o modelo
test_score_linear = []
test_score_polynomial = []
for sample in samples:
  # Criando variáveis X, Y para regressão linear
  X_test = sample[['dummy_smoker']].to_numpy()
  Y_test = sample['charges'].to_numpy()
  test_score_linear.append(model_linear.score(X_test, Y_test))

  # Criando variável polinomial para o X
  X_test_poly = poly_features.fit_transform(X_test)
  test_score_polynomial.append(model_poly.score(X_test_poly, Y_test))

print(test_score_linear)
print(test_score_polynomial)

[0.4792659846752161, 0.46163208333164263, 0.6960269091694877, 0.6876963492540012, 0.586069655094251, 0.6122848365185185, 0.6178845447800677, 0.5907436926252937, 0.7256686203914688, 0.6791142340807754]
[0.4945191678397579, 0.4743202927003427, 0.6901319056268227, 0.6803339734704803, 0.5987497923594944, 0.6179998437605176, 0.6132336550636235, 0.5927713985555697, 0.7127161274385241, 0.677195139438479]


In [69]:
alpha = 0.1
# Modelagem do teste de hipóteses
# H0: score de teste é igual ao score de treino
#     a minha regressão não perde poder de explicação quando testa novas variáveis
# H1: score de teste é diferente do score de treino
#     a minha regressão não explica a variável de interesse da mesma maneira que o teste.

t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
print(score_linear)
print(p_valor)
if p_valor < alpha:
  print("Regressão Linear: Aceito H1 e rejeito H0")
else:
  print("Regressão Linear: Aceito H0 e rejeito H1")

t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)
print(score_polynomial)
print(p_valor)
if p_valor < alpha:
  print("Regressão Polinomial: Aceito H1 e rejeito H0")
else:
  print("Regressão Polinomial: Aceito H0 e rejeito H1")

0.6250613522379534
0.6949024206426823
Regressão Linear: Aceito H0 e rejeito H1
0.6235990126892539
0.7490880139509848
Regressão Polinomial: Aceito H0 e rejeito H1
